# Clustering and Visualising Documents using Word Embeddings 

This can be [run in Google Colab](https://colab.research.google.com/github/jreades/ph-word-embeddings/blob/main/Clustering_Word_Embeddings.ipynb). However, this tutorial was written on a (virtualised) system which had access to 4 CPUs, 8GB of RAM, and a Solid State Disk (SSD) drive. We have included *rough* timings for each step based on those computing resources. You can expect the Colab notebook to be at least 50% slower and it may _crash_ when you get to hierarchical clustering. If that happens to you, we've provided code to subsample the data so that you aren't clustering quite so many observations.

Generally useful libraries.

In [ ]:
import pandas as pd
import numpy as np
import feather
import random
import math
import ast
import os
import re

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm

import seaborn as sns

Needed to calculate the word embeddings.

In [ ]:
from gensim.models.word2vec import Word2Vec

Needed for the dimensionality reduction stage.

In [ ]:
try:
    import umap
except ModuleNotFoundError:
    !pip install umap-learn
    import umap

Needed for hierarchical clustering stage.

In [ ]:
try:
    from kneed import KneeLocator
except ModuleNotFoundError:
    !pip install kneed
    from kneed import KneeLocator

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster, centroid
from sklearn.metrics import silhouette_score, silhouette_samples
from tabulate import tabulate
import pickle

Needed for the validation and visualisation stage.

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from wordcloud import WordCloud

The code below tries to find a narrow sans-serif TTF font by path that is slightly nicer than the default for the WordCloud library. You would need to update this default for your own system. You can list available fonts using (/ht [imsc](https://stackoverflow.com/a/8755818/4041902)):
```python
import matplotlib.font_manager
matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
```

In [ ]:
import matplotlib.font_manager
fonts = matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
fp = fonts[0] # Ensure at least _something_ is set here
for f in fonts:
    if 'LiberationSansNarrow-Regular' in f:
        fp = f
        break
    elif 'SansNarrow' in f:
        fp = f
print(f"Using font: {fp}")

## Configuration

In [ ]:
# Random seed
rs = 42

# Whether to load a cached model result
# to allow further exploration of model
# parameters later.
cache = True

## Load

<div style="border:dotted 3px red; padding: 15px; background-color: rgb(255,225,225);">
    This cleaned file needs a permanent home that is not my personal site. But also want to ensure BL gets credit for providing the original resource. They might be fine hosting this. Will ask them this week. They can also host the source file, which is a subset of their EThOS resource with DDCs appended.
</div>

In [ ]:
# Name of the file
fn = 'ph-tutorial-data-cleaned.csv.gz'

# See if the data has already been downloaded, and
# if not, download it from the web site. We save a
# copy locally so that you can run this tutorial
# offline and also spare the host the bandwidth costs
if os.path.exists(os.path.join('data',fn)):
    df = pd.read_csv(os.path.join('data',fn), low_memory=False, encoding="ISO-8859-1").set_index('EThOS_ID')
else:
    # We will look for/create a 'data' directory
    if not os.path.exists('data'):
        os.makedirs('data')
   
    # Download and save
    df = pd.read_csv(f'http://www.reades.com/{fn}', low_memory=False, encoding="ISO-8859-1").set_index('EThOS_ID')
    df.to_csv(os.path.join('data',fn), compression='gzip')

In [ ]:
df['tokens'] = df.tokens.apply(ast.literal_eval)

## Dimensionality Reduction

While I'm confident about the output from UMAP in a _general_ sense, I'm much less certain about the default _distance_ measure (see [sklearn docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html)). I wonder if the metric should be `cosine`? [This article](https://towardsdatascience.com/9-distance-measures-in-data-science-918109d069fa) appears to offer some help, but points on to a [longer discussion](https://stats.stackexchange.com/questions/99171/why-is-euclidean-distance-not-a-good-metric-in-high-dimensions) where `manhattan` is argued to be a good representation.

UMAP offers a very wide range of distance metrics:

- Minkowski style metrics
  - euclidean
  - manhattan
  - chebyshev
  - minkowski
- Miscellaneous spatial metrics
  - canberra
  - braycurtis
  - haversine
- Normalized spatial metrics
  - mahalanobis
  - wminkowski
  - seuclidean
- Angular and correlation metrics
  - cosine
  - correlation

### Functions

In [ ]:
# Assumes that there is a column that contains the 
# document embedding as an array/list that needs to be 
# extracted to a new data frame
def x_from_df(df:pd.DataFrame, col:str='Embedding') -> pd.DataFrame:
    cols = ['E'+str(x) for x in np.arange(0,len(df[col].iloc[0]))]
    return pd.DataFrame(df[col].tolist(), columns=cols, index=df.index)

### Configure Process

In [ ]:
dmeasure = 'euclidean'
rdims    = 4 # r-dims == Reduced dimensionality
print(f"UMAP dimensionality reduction to {rdims} dimensions with '{dmeasure}' distance measure.")

### Reduce Dimensionality

Expect this to take **about 1 minute** (or **2.5 minutes on Google Collab**).

In [ ]:
%%time 
X = x_from_df(df, col='word_vec')
    
reducer = umap.UMAP(
    n_neighbors=25,
    min_dist=0.01,
    n_components=rdims,
    random_state=rs)
    
# Basically reduces our feature vectors for each thesis, down to n dimensions
X_embedded = reducer.fit_transform(X)

### Merge on to Data

This next block turns the output Numpy array into a data frame with one column for each reduced dimension.

In [ ]:
embedded_dict = {}
for i in range(0,X_embedded.shape[1]):
    embedded_dict[f"Dim {i+1}"] = X_embedded[:,i] # D{dimension_num} (Dim 1...Dim n)

# dfe == df embedded
dfe = pd.DataFrame(embedded_dict, index=df.index)
del(embedded_dict)

dfe.head(3)

Merge the projection on to the main data frame so that we can easily explore the results.

In [ ]:
projected = df.join(dfe).sort_values(by=['ddc1','ddc2'])
print(projected.columns.values)
projected.head(3)

### Plot Embeddings by DDC

We plot these for reference, but don't save them as we spend more time tweaking the outputs in notebook 7 so we can do it for all outputs at the same time.

In [ ]:
f, axs = plt.subplots(1,2,figsize=(14,6))
axs = axs.flatten()

sns.scatterplot(data=projected, x='Dim 1', y='Dim 2', hue='ddc1', s=5, alpha=0.1, ax=axs[0]);
axs[0].axis('off')
axs[0].set_title('DDC1 Group')
axs[0].get_legend().set_title("")

sns.scatterplot(data=projected, x='Dim 1', y='Dim 2', hue='ddc2', s=5, alpha=0.1, ax=axs[1]);
axs[1].axis('off')
axs[1].set_title('DDC2 Group')
axs[1].get_legend().set_title("");
#plt.savefig(os.path.join('data','DDC_Plot.png'), dpi=150)
plt.show()

### Intermediate Save

In [ ]:
projected.reset_index().to_feather(os.path.join('data','ph-tutorial-data-final.feather'))

## Hierarchical Clustering

### Cluster

Expect this next stage to take **about 4 minutes** (or **8 minutes on Google Collab**). Using Google Collab I experienced a crash at this point which is why we have the intermediate saves above. You can reload the data from the feather even when the session ran out of RAM and died. If restarting from this point still produces Out-of-Memory errors than you may need to sample the data instead:
```python
projected = pd.read_feather(os.path.join('data','ph-tutorial-data-final.feather')).set_index('EThOS_ID').sample(frac=0.5)
```
When you perform the `join` later the unsampled records should fall out naturally though, obviously, your results will begin to differ substantially from the ones presented in the tutorial.

In [ ]:
%%time 

Z = linkage(projected[[x for x in projected.columns if x.startswith('Dim ')]], method='ward', metric='euclidean')

### Intermediate Save

In [ ]:
pickle.dump(Z, open(os.path.join('data','Z.pickle'), 'wb'))

#### Z-Matrix Output

`Z` is a $(n-1)$ by 4 matrix. At the $i$-th iteration, clusters with indices $Z[i, 0]$ and $Z[i, 1]$ are combined to form cluster $n+i$. A cluster with an index less than $n$ corresponds to one of the $n$ original observations. The distance between clusters $Z[i, 0]$ and $Z[i, 1]$ is given by $Z[i, 2]$. The fourth value $Z[i, 3]$ represents the number of original observations in the newly formed cluster.

In [ ]:
table = []

# Take the 1st, 10000th, '-2000th', and '-1st' observations
for i in [0, 10000, -2000, -1]:
    r = list(Z[i])
    r.insert(0,(i if i >= 0 else len(Z)+i))
    table.append(r)
    table[-1][1] = int(table[-1][1])
    table[-1][2] = int(table[-1][2])
    table[-1][4] = int(table[-1][4])

display(
    tabulate(table, 
             headers=["Iteration","$c_{i}$","$c_{j}$","$d_{ij}$","$\sum{c_{i},c_{j}}$"], 
             floatfmt='0.3f', tablefmt='html'))

### Show Top _n_ Clusters

In [ ]:
last_cls = 100 # The number of last clusters to show in the dendogram

plt.title(f'Hierarchical Clustering Dendrogram (truncated at {last_cls} clusters)')
plt.xlabel('Sample Index (includes count of records in cluster)')
plt.ylabel('Distance')
fig = plt.gcf()
fig.set_size_inches(20, 7)
fig.set_dpi(300)
dendrogram(
    Z,
    truncate_mode='lastp', # truncate dendrogram to the last p merged clusters
    p=last_cls,            # and set a value for last p merged clusters
    show_leaf_counts=True, # if parentheses then this is a count of observations, otherwise an id
    leaf_rotation=90.,
    leaf_font_size=8.,
    show_contracted=False, # to get a distribution impression in truncated branches
)
#plt.savefig(os.path.join('data',f'Dendogram-{c.dmeasure}-{last_cls}.png'))
plt.show()

### Silhouette Scoring

This process is slow and computationally intensive: it is calculating a silhouette score for every clustering option between `start` and `end` to give you a sense of how the silhouette score evolves with the number of clusters. A falling silhouette score is normal since, the smaller the number of observations in the cluster, the more you're likely to see some badly-bitted observations within a cluster... at least up until the point where you start having very small clusters indeed. What we're going to be looking for is the 'knee' where this process levels out.

You're looking at **_over_ 10 seconds per clustering**, IIRC, so for 50 clusters that's **about 10 minutes**.

In [ ]:
%%time 

start_cl = 2
end_cl   = 50

sil_scores = []

print("Scoring cluster levels: ", end="")

X_embedded = projected[[x for x in projected if x.startswith('Dim ')]]

for i in range(start_cl,end_cl):
    print(".", end="")
    clusterings = fcluster(Z, i, criterion='maxclust')
    
    # Calculate silhouett average
    sil_avg = silhouette_score(X_embedded, clusterings)
    
    # Append silhouette scores
    sil_scores.append(sil_avg)

print("\nDone.")

### Scree Plot

Using the silhouette scores calculatee above we can now generate a scree plot.

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,5));
sns.lineplot(x=np.arange(start_cl,start_cl+len(sil_scores)), y=sil_scores, ax=ax)
ax.set_title("")
ax.set_xlabel("Number of Clusters")
ax.set_ylabel("Average Silhouette Score")

decreasing = True
last_sil   = 1.0
selected   = 0

for i in range(1,len(sil_scores)):
    if sil_scores[i] < last_sil and decreasing:
        #print(f"Decreasing {i}: {sil_scores[i]:0.5f}")
        last_sil = sil_scores[i]
    elif sil_scores[i] > last_sil and decreasing:
        #print(f"Increasing {i}: {sil_scores[i]:0.5f}")
        last_sil = sil_scores[i]
        decreasing = False
    elif sil_scores[i] > last_sil and not decreasing:
        #print(f"Increasing {i}: {sil_scores[i]:0.5f}")
        last_sil = sil_scores[i]
        selected = i

ax.vlines(x=selected+start_cl, ymin=np.array(sil_scores).min(), ymax=np.array(sil_scores).max(), color='red', linestyle='dotted')
plt.text(x=selected+start_cl+1, y=np.array(sil_scores).max()/2, s=f'Suggest $c$={selected+start_cl}')

plt.suptitle(f"Scree Plot for Hierarchical Clustering", fontsize=14);
#plt.savefig(os.path.join('data',f'Silhouette-Scree.png'), dpi=150)
plt.show();

### Knee Locator 

We can eyeball the scree plot, but a good sanity-check is to use the [kneed](https://pypi.org/project/kneed/) utility to automate the process. Depending on how your clusters score this may or may not be helpful: _e.g._ sometimes a relatively small deviation triggers the 'knee' when that is obviously only a 'blip'.

In [ ]:
kn = KneeLocator(np.arange(3,3+len(sil_scores)), sil_scores, curve="convex", direction="decreasing")
print(f'Suggest c={kn.knee}')

## Investigating Results

We're now going to investigate the clustering results in greater detail.

### Functions

In [ ]:
def label_clusters(src_df:pd.DataFrame, clusterings:np.ndarray, ddc_level:int=1):
    
    num_clusters = clusterings.max()
    
    tmp = pd.DataFrame({f'Cluster_{num_clusters}':clusterings}, index=src_df.index)
    
    joined_df = src_df.join(tmp, how='inner')
    
    labels = get_dominant_cat(joined_df, clusterings.max(), ddc_level)
    joined_df[f'Cluster_Name_{num_clusters}'] = joined_df[f'Cluster_{num_clusters}'].apply(lambda x: labels[x])

    return joined_df

def plt_silhouette(src_df:pd.DataFrame, clusterings:np.ndarray) -> plt:
    
    num_clusters = clusterings.max()
    sample_silhouette_values = silhouette_samples(src_df, clusterings)
    
    scale = cm.get_cmap(get_scale_nm(num_clusters)).colors
        
    fig, ax = plt.subplots(1,1,figsize=(10,7));
    y_lower = 10
    mx = clusterings.min()

    for cl in range(1,clusterings.max()+1):

        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them. 
        ith_cluster_silhouette_values = sample_silhouette_values[clusterings==cl]
        ith_cluster_silhouette_values.sort() # Note, returns None!
        y_upper = y_lower + ith_cluster_silhouette_values.shape[0]

        ax.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=scale[cl],
            edgecolor=scale[cl],
            alpha=1.0,
        )

        # Label the silhouette plots with their cluster numbers at the middle
        #ax.text(-0.05, y_lower + 0.5 * ith_cluster_silhouette_values.shape[0], str(c))
        ax.annotate(f'Cluster {cl}',
            xy=(np.min(sample_silhouette_values), y_lower + 0.5 * ith_cluster_silhouette_values.shape[0]), 
            textcoords='data', horizontalalignment='left', verticalalignment='top')

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax.set_title(f"Silhouette Plot for {num_clusters} Clusters", fontsize=14);
    ax.set_xlabel("Silhouette coefficient values")
    ax.set_ylabel("")
    ax.set_yticks([])  # Clear the yaxis labels / ticks

    return ax

def get_dominant_cat(clustered_df:pd.DataFrame, num_clusters:int, ddc_level:int=1):
    labels = {}
    struct = {}

    # First, work out the dominant group in each cluster
    # and note that together with the cluster number --
    # this gives us a dict with key==dominant group and 
    # then one or more cluster numbers from the output
    # above.
    for cl in range(1,num_clusters+1):
    
        # Identify the dominant 'domain' (i.e. group by
        # DDC description) using the value counts result.
        dom     = clustered_df[clustered_df[f'Cluster_{num_clusters}']==cl][f'ddc{ddc_level}'].value_counts().index[0]
        print(f"Cluster {cl} dominated by {dom} theses.")
    
        if struct.get(dom) == None:
            struct[dom] = []
    
        struct[dom].append(cl)

    # Next, flip this around so that we create useful
    # cluster labels for each cluster. Since we can have
    # more than one cluster dominated by the same group
    # we have to increment them (e.g. History 1, History 2)
    for g in struct.keys():
        if len(struct[g])==1:
            labels[struct[g][0]]=g
            #print(f'{g} maps to Cluster {struct[g][0]}')
        else:
            for s in range(0,len(struct[g])):
                labels[struct[g][s]]=f'{g} {s+1}'
                #print(f'{g} {s+1} maps to Cluster {struct[g][s]}')
    return labels

def get_scale_nm(num_clusters:int):
    if num_clusters <= 10:
        return 'tab10'
    elif num_clusters <= 20:
        return 'tab20'
    else:
        print("More than 20 clusters, this is hard to render meaningfully!")
        #cmap = mcolors.LinearSegmentedColormap.from_list("", ["indigo","gold"], gamma=0.5, N=num_clusters)
        #cmap = mcolors.ListedColormap.from_list("", ["indigo","gold"], gamma=0.5, N=num_clusters)
        cmap = cm.get_cmap('Spectral', num_clusters)
        return cmap(np.linspace(0,1,cmap.N))

### 2 Clusters

In [ ]:
num_clusters = 2
ddc_level = 1

In [ ]:
# Extract clustering based on Z object
clusterings  = fcluster(Z, num_clusters, criterion='maxclust')

# Plot silhouette
ax = plt_silhouette(projected[[x for x in projected.columns if x.startswith('Dim ')]], clusterings)
f = ax.get_figure()
#plt.savefig(os.path.join('data',f'Silhouette-c{num_clusters}.png'), dpi=150)
plt.show();

# Label clusters and add to df
clustered_df = label_clusters(projected, clusterings, ddc_level=ddc_level)

# Diagnostics
print()

# Classification report gives a (statistical) sense of power (TP/TN/FP/FN)
print(classification_report(clustered_df[f'ddc{ddc_level}'], clustered_df[f'Cluster_Name_{num_clusters}']))

# A confusion matrix is basically a cross-tab (without totals, which I think are nice to add)
pd.crosstab(columns=clustered_df[f'Cluster_Name_{num_clusters}'], 
            index=clustered_df[f'ddc{ddc_level}'], 
            margins=True, margins_name='Total')


### 4 Clusters

In [ ]:
num_clusters = 4
ddc_level = 2

In [ ]:
# Extract clustering based on Z object
clusterings  = fcluster(Z, num_clusters, criterion='maxclust')

# Plot silhouette
ax = plt_silhouette(projected[[x for x in projected.columns if x.startswith('Dim ')]], clusterings)
f = ax.get_figure()
#plt.savefig(os.path.join('data',f'Silhouette-c{num_clusters}.png'), dpi=150)
plt.show();

# Label clusters and add to df
clustered_df = label_clusters(projected, clusterings, ddc_level=ddc_level)

# Diagnostics
print()

# Classification report gives a (statistical) sense of power (TP/TN/FP/FN)
print(classification_report(clustered_df[f'ddc{ddc_level}'], clustered_df[f'Cluster_Name_{num_clusters}']))

# A confusion matrix is basically a cross-tab (without totals, which I think are nice to add)
pd.crosstab(columns=clustered_df[f'Cluster_Name_{num_clusters}'], 
            index=clustered_df[f'ddc{ddc_level}'], 
            margins=True, margins_name='Total')


### Selected _n_ Clusters

In [ ]:
num_clusters = 15
ddc_level = 3

In [ ]:
# Extract clustering based on Z object
clusterings  = fcluster(Z, num_clusters, criterion='maxclust')

# Plot silhouette
ax = plt_silhouette(projected[[x for x in projected.columns if x.startswith('Dim ')]], clusterings)
f = ax.get_figure()
#plt.savefig(os.path.join('data',f'Silhouette-c{num_clusters}.png'), dpi=150)
plt.show();

# Label clusters and add to df
clustered_df = label_clusters(projected, clusterings, ddc_level=ddc_level)

# Diagnostics
print()

# If you have a situation where multiple clusters have been detected within 
# a DDC (e.g. Biochemistry 1, 2, 3) then you need to combine these back into
# Biochemsitry for the classification and crosstab or you'll get zero errors
# because you couldn't have predicted Biochemistry 2 from Biochemsitry...
sddc = clustered_df[f'ddc{ddc_level}']
scl  = clustered_df[f'Cluster_Name_{num_clusters}']

keys = sorted(scl.unique())
vals = [re.sub(' \d+$','',x) for x in keys]
mapping = dict(zip(keys, vals))

# Classification report gives a (statistical) sense of power (TP/TN/FP/FN)
print(classification_report(sddc, scl.map(mapping), zero_division=1))

# A confusion matrix is basically a cross-tab (without totals, which I think are nice to add)
pd.crosstab(columns=scl.map(mapping), index=sddc, margins=True, margins_name='Total')

## Misclassifications

Here we are trying to look in more detail at the PhDs that have (potentially!) been 'misclassified' by the experts--our clustering places them in a different group from the one specified by the DDC. Clearly, we'll have some false-positives in here as well, but the point is to examine the degree to which misclassification is both plausible and useful in terms of demonstrating the value of the NLP approach.

In [ ]:
# This makes it easier to read the thesis titles in the output
pd.set_option('display.max_colwidth',150)

### Configure Process

In [ ]:
stopw = ['study','examine','analysis','system','use','design','model','data','within']

#### Recluster

Not needed if the `num_clusters` and `ddc_level` are the same as you ran above.

In [ ]:
num_clusters = 4
ddc_level    = 2

# Extract clustering based on Z object
clusterings  = fcluster(Z, num_clusters, criterion='maxclust')

# Label clusters and add to df
clustered_df = label_clusters(projected, clusterings, ddc_level=ddc_level)

#### Merge with Intermediate Save

We need the tokens again...

In [ ]:
df = pd.read_feather(os.path.join('data','ph-tutorial-data-embeddings.feather')).set_index('EThOS_ID')

In [ ]:
# Full Data Frame (fdf)
fdf = df.join(clustered_df, rsuffix='_dupe')
fdf.drop(columns=[x for x in fdf.columns if x.endswith('_dupe')], inplace=True)
fdf.columns

### Find Misclassified Theses

This approach to misclassification works well for level 1 and level 2 of the DDC, but it gets a lot more complex when you're looking at level 3 because there are _so_ many different groups and misclassifications (e.g. Economics vs Financial Economics) that the results become much harder to interpret.

In [ ]:
# This will be based on whatever clustering above you ran *last* (ncls/ddc_level)
# misc = mis-classified records
misc = fdf[fdf[f'ddc{ddc_level}'] != fdf[f'Cluster_Name_{num_clusters}']]
cols = ['Title',f'ddc{ddc_level}',f'Cluster_Name_{num_clusters}']
misc.sample(5, random_state=rs)[cols]

Below is the distribution of 'misclassified' theses by DDC group:

In [ ]:
print(f"There are {misc.shape[0]:,} ({(misc.shape[0]/fdf.shape[0])*100:0.1f}%) 'misclassified' theses.")
print()
misc.groupby(by=f'ddc{ddc_level}')[f'Cluster_Name_{num_clusters}'].value_counts()

### Word Clouds

This approach is less technically sophisticated and robust than the one set out in Maarten Grootendorst's [CTFIDF](https://github.com/MaartenGr/cTFIDF/blob/master/ctfidf.py) module (as developed in [topic modelling with BERT](https://towardsdatascience.com/topic-modeling-with-bert-779f7db187e6) and [class-based TF/IDF](https://towardsdatascience.com/creating-a-class-based-tf-idf-with-scikit-learn-caea7b15b858)), but it saves having to install _another_ module and produces output that is easier to align with the needs of the WordCloud library.

In [ ]:
pd.options.display.float_format = "{:,.3f}".format

tfidfs = {}

vec = TfidfVectorizer(use_idf=True, ngram_range=(1,1), smooth_idf=True, stop_words=stopw)

for d in fdf[f'ddc{ddc_level}'].unique():
    
    print(f"Examining {d} DDC")
    tfidfs[d] = []
    
    # All records classified under this DDC
    ddc_df = fdf[fdf[f'ddc{ddc_level}']==d].copy()
    
    # Those records that are part of this DDC
    # but were clustered with *another* group
    # going by the dominant class in that cluster.
    sub_df = misc[misc[f'ddc{ddc_level}']==d].copy()
    
    print(f"  ddc_df: {ddc_df.shape[0]:>7,}")
    print(f"  sub_df: {sub_df.shape[0]:>7,}")
    print(f"  remain: {ddc_df[~ddc_df.index.isin(misc.index)].shape[0]:>7,}")
    
    print(f"  {(sub_df.shape[0]/ddc_df.shape[0])*100:0.1f}% of {d} PhDs were clustered in other disciplines.")
    
    # This removes the 'Earth Sc. 2', 'Earth Sc. 1' distinction for example.
    # You would normally only encounter this working with DDC3.
    sub_df.loc[:,'Cluster Name'] = sub_df[f'Cluster_Name_{num_clusters}'].str.replace("\s\d+$","",regex=True)
    
    # Convert tokens back to string
    # And fit the corpus using IDF
    corpus  = ddc_df.tokens.str.join(' ').fillna(' ').values 
    vec.fit(corpus)
    
    # One image per DDC Category
    f,axes = plt.subplots(1, len(sub_df['Cluster Name'].unique()), figsize=(14,5))
    
    for i, cl in enumerate(sub_df['Cluster Name'].unique()):
        
        sub_cdf = sub_df[sub_df['Cluster Name']==cl]
        print(f"  PhDs classified as {cl} ({sub_cdf.shape[0]:,})")
        
        tcorpus = vec.transform(sub_cdf.tokens.str.join(' ').fillna(' ').values)
        
        tfidf   = pd.DataFrame.from_records(tcorpus.toarray(), index=sub_cdf.index, columns=vec.get_feature_names_out())
        tfterms = tfidf.T.sum(axis=1)
        
        tfidfs[d].append(
            pd.DataFrame(
              {f"{cl} Term":  tfterms.sort_values(ascending=False).index, 
               f"{cl} Value": tfterms.sort_values(ascending=False).values}
            )
        )
        
        #print(tfterms.sort_values(ascending=False).head(5))
        #print()
        
        Cloud = WordCloud(background_color=None, mode='RGBA', relative_scaling=0.5, font_path=fp)
        
        ax = axes.flatten()[i]
        ax.set_title(f"{d} clustered with {cl} ($n$={sub_cdf.shape[0]:,})")
        ax.imshow(Cloud.generate_from_frequencies(tfterms))
        ax.axis("off")
    
    plt.tight_layout()
    #plt.savefig(os.path.join('images',f"DDC_Cloud-c{num_clusters}-ddc{d}-tfidf.png"), dpi=150)
    plt.show()
        
print("Done.")

### 15-Cluster Analysis

In [ ]:
num_clusters = 15
ddc_level    = 3

In [ ]:
# Extract clustering based on Z object
clusterings  = fcluster(Z, num_clusters, criterion='maxclust')

# Label clusters and add to df
clustered_df = label_clusters(projected, clusterings, ddc_level=ddc_level)

# Load up intermediate save
df = pd.read_feather(os.path.join('data','ph-tutorial-data-embeddings.feather')).set_index('EThOS_ID')

# Join up with tokens
fdf = df.join(clustered_df, rsuffix='_dupe')
fdf.drop(columns=[x for x in fdf.columns if x.endswith('_dupe')], inplace=True)
fdf.columns

In [ ]:
cls  = [x for x in fdf[f'Cluster_Name_{num_clusters}'].unique() if isinstance(x,str)] # Catch unexpected unlabelled result
print(f"Found clusters: {', '.join(sorted(cls))}.")
cols = 3
rows = math.ceil(len(cls)/cols)

vec = TfidfVectorizer(use_idf=True, ngram_range=(1,1), smooth_idf=True, stop_words=stopw)

# We only need to calculate this once since 
# we're comparing to the full corpus
idf_df  = fdf
corpus  = idf_df.tokens.str.join(' ').fillna(' ').values # Convert tokens back to string
vec.fit(corpus) # And fit the corpus using IDF

f,axs = plt.subplots(rows, cols, figsize=(cols*5, rows*5))

for i,v in enumerate(sorted(cls)):
    
    # Truncate cluster long name (cl.nm) just in case
    if len(v) > 26:
        cln = f"{v[:26]}..."
    else:
        cln = v
    ax = axs.flatten()[i]
    
    print(f"Cluster name is '{cln}'.")
    
    # The TF is from the selected cluster
    tf_df  = fdf[fdf[f'Cluster_Name_{num_clusters}']==v]
    
    # Share of corpus
    corpus_share = f"{(tf_df.shape[0]/idf_df.shape[0])*100:0.2f}"
    print(f"  Cluster contains {tf_df.shape[0]:,} ({corpus_share}% of records)")
    
    # Transform the selected cluster documents
    tcorpus = vec.transform(tf_df.tokens.str.join(' ').fillna(' ').values)
    
    # And get the words back out in a format that
    # we can use for a word cloud
    tfidf   = pd.DataFrame.from_records(tcorpus.toarray(), index=tf_df.index, columns=vec.get_feature_names_out())
    tfterms = tfidf.T.sum(axis=1)
    
    Cloud = WordCloud(background_color=None, mode='RGBA', 
                      width=int(ax.bbox.width), height=int(ax.bbox.height),
                      relative_scaling=0.5, font_path=fp, 
                      max_words=50, random_state=rs)
    
    #print(f"width={int(ax.bbox.width)}, height={int(ax.bbox.height)}")
    
    ax.title.set_text(f"{cln} ({corpus_share}% of docs)")
    ax.set_axis_off()
    ax.imshow(Cloud.generate_from_frequencies(tfterms))
    print("  Done.")

#plt.suptitle(f"Distinctive Words for Clusters")
plt.axis("off")
plt.tight_layout()
plt.savefig(os.path.join('data',f"Word_Cloud-c{num_clusters}-tfidf.png"), dpi=150)

print("Done.")